In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install tensorflow

In [ ]:
import os, numpy as np, pandas as pd, tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import load_img, img_to_array

IMG_SIZE = 300
BATCH_SIZE = 16

BASE_DIR = "/content/drive/MyDrive/BrainTumorDetection"

LOG_DIR   = f"{BASE_DIR}/Logs"
MODEL_DIR = f"{BASE_DIR}/Models"

os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

In [ ]:
@tf.keras.utils.register_keras_serializable()
class FocalLoss(tf.keras.losses.Loss):
    def __init__(self, gamma=2.0, alpha=0.25,
                 reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE,
                 name="focal_loss"):
        super().__init__(reduction=reduction, name=name)
        self.gamma = gamma
        self.alpha = alpha

    def call(self, y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
        ce = -y_true * tf.math.log(y_pred)
        fl = self.alpha * tf.pow(1 - y_pred, self.gamma) * ce
        return tf.reduce_mean(tf.reduce_sum(fl, axis=1))

    def get_config(self):
        config = super().get_config()
        config.update({
            "gamma": self.gamma,
            "alpha": self.alpha
        })
        return config

In [ ]:
@tf.keras.utils.register_keras_serializable()
def attention_block(x):
    gap = layers.GlobalAveragePooling2D()(x)
    dense = layers.Dense(x.shape[-1]//8, activation="relu")(gap)
    scale = layers.Dense(x.shape[-1], activation="sigmoid")(dense)
    scale = layers.Reshape((1,1,x.shape[-1]))(scale)
    return layers.Multiply()([x, scale])

In [ ]:
import pandas as pd
import os

def last_epoch(csv_path):
    # File does not exist
    if not os.path.exists(csv_path):
        return 0

    # File exists but is empty
    if os.path.getsize(csv_path) == 0:
        return 0

    try:
        df = pd.read_csv(csv_path)
        return len(df)
    except Exception:
        # Covers EmptyDataError or corrupted CSV
        return 0

In [ ]:
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input


class Stage2MRIGenerator(Sequence):
    def __init__(
        self,
        root_dir,
        batch_size=16,
        img_size=(300, 300),
        augment=False,
        shuffle=True
    ):
        super().__init__()

        self.samples = []
        self.batch_size = batch_size
        self.img_size = img_size
        self.augment = augment
        self.shuffle = shuffle

        # class folders: glioma, meningioma, pituitary
        self.classes = sorted([
            d for d in os.listdir(root_dir)
            if os.path.isdir(os.path.join(root_dir, d))
        ])
        self.class_map = {c: i for i, c in enumerate(self.classes)}

        for cls in self.classes:
            cls_dir = os.path.join(root_dir, cls)
            for img in os.listdir(cls_dir):
                if img.lower().endswith((".jpg", ".jpeg", ".png")):
                    self.samples.append(
                        (os.path.join(cls_dir, img), self.class_map[cls])
                    )

        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.samples) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.samples)

    def __getitem__(self, idx):
        batch = self.samples[
            idx * self.batch_size : (idx + 1) * self.batch_size
        ]

        X, y = [], []

        for path, label in batch:
            img = load_img(path, target_size=self.img_size)
            img = img_to_array(img)

            # EfficientNet preprocessing
            img = preprocess_input(img)

            # 🎯 Class-specific augmentation
            if self.augment:
                # Glioma = stronger augmentation
                if self.classes[label] == "glioma":
                    if np.random.rand() > 0.3:
                        img = tf.image.flip_left_right(img)
                    if np.random.rand() > 0.3:
                        img = tf.image.random_brightness(img, 0.15)
                else:
                    if np.random.rand() > 0.6:
                        img = tf.image.flip_left_right(img)

            X.append(img)
            y.append(label)

        return (
            np.array(X, dtype=np.float32),
            tf.keras.utils.to_categorical(y, len(self.classes))
        )

In [ ]:
def build_stage2_model(n):
    base = EfficientNetB3(weights="imagenet", include_top=False,
                          input_shape=(IMG_SIZE, IMG_SIZE, 3))
    base.trainable = False

    x = attention_block(base.output)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(256, activation="relu")(x)
    out = layers.Dense(n, activation="softmax")(x)

    return models.Model(base.input, out)

In [ ]:
if not os.path.exists(STAGE2_LOG):
    with open(STAGE2_LOG, "w") as f:
        f.write("")

In [ ]:
initial_epoch = last_epoch(STAGE2_LOG)
print("Stage-2 resume from:", initial_epoch)

if os.path.exists(STAGE2_BEST):
    model2 = tf.keras.models.load_model(
        STAGE2_BEST,
        custom_objects={"FocalLoss": FocalLoss, "attention_block": attention_block}
    )
else:
    model2 = build_stage2_model(3)

model2.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=FocalLoss(),
    metrics=["accuracy"]
)

callbacks2 = [
    ModelCheckpoint(STAGE2_BEST, save_best_only=True, monitor="val_loss"),
    CSVLogger(STAGE2_LOG, append=True),
    ReduceLROnPlateau(patience=3, factor=0.3),
    EarlyStopping(patience=6, restore_best_weights=True)
]

model2.fit(
    Stage2MRIGenerator(STAGE2_TRAIN, augment=True),
    validation_data=Stage2MRIGenerator(STAGE2_VAL),
    epochs=40,
    initial_epoch=initial_epoch,
    callbacks=callbacks2
)

model2.save(STAGE2_FINAL)